# 针对原始数据进行特征工程

In [205]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest

# 导入数据清洗规则

载入数据

In [206]:
#载入数据
data_train = pd.read_csv('./数据集/train_data.csv')
# data_train['Type'] = 'Train'
data_test = pd.read_csv('./数据集/test_a.csv')
# data_test['Type'] = 'Test'
target_test = pd.read_csv('./数据集/评分文件/sub_a_913.csv')
# data_all = pd.concat([data_train, data_test], ignore_index=True)

In [207]:
target_test.head()

,pre
0,4808
1,4770
2,5701
3,3635
4,4387


数据清洗

In [208]:
def clean1(data_all):
    data_all['rentType'][data_all['rentType'] == '--'] = '未知方式'

    # 将buildYear列转换为整型数据
    buildYearmean = pd.DataFrame(data_all[data_all['buildYear'] != '暂无信息']['buildYear'].mode())
    data_all.loc[data_all[data_all['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearmean.iloc[0, 0]
    data_all['buildYear'] = data_all['buildYear'].astype('int')

    # 处理pv和uv的空值
    data_all['pv'].fillna(data_all['pv'].mean(), inplace=True)
    data_all['uv'].fillna(data_all['uv'].mean(), inplace=True)
    data_all['pv'] = data_all['pv'].astype('int')
    data_all['uv'] = data_all['uv'].astype('int')

    # 时间字段处理
    def month(x):
            month = int(x.split('/')[1])
            return month
    def day(x):
        day = int(x.split('/')[2])
        return day
    data_all['month'] = data_all['tradeTime'].apply(lambda x: month(x))
    data_all['day'] = data_all['tradeTime'].apply(lambda x: day(x))

    # 删除无关字段
    data_all.drop('city', axis=1, inplace=True)
    data_all.drop('tradeTime', axis=1, inplace=True)
    data_all.drop('ID', axis=1, inplace=True)
    
    return data_all
    
data_train = clean1(data_train)
data_test = clean1(data_test)
# # 转换object类型数据
# columns = ['rentType','communityName','houseType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate']

# for feature in columns:
#     data_all[feature] = LabelEncoder().fit_transform(data_all[feature])
    


In [209]:
# 异常值处理
IForest = IsolationForest(contamination=0.01)
IForest.fit(data_train["tradeMoney"].values.reshape(-1,1))
y_pred = IForest.predict(data_train["tradeMoney"].values.reshape(-1,1))
drop_index = data_train.loc[y_pred==-1].index
print(drop_index)
data_train.drop(drop_index,inplace=True)

# 丢弃部分异常值
data_train = data_train[data_train.area <= 200]
data_train = data_train[(data_train.tradeMoney <=16000) & (data_train.tradeMoney >=700)]
data_train.drop(data_train[(data_train['totalFloor'] == 0)].index, inplace=True)

# 不同region 下的area 与TradeMoney shu
def cleanData(data):
    data.drop(data[(data['region']=='RG00001') & (data['tradeMoney']<1000)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>250)&(data['tradeMoney']<20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00002') & (data['area']<100)&(data['tradeMoney']>60000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']<300)&(data['tradeMoney']>30000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<1500)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<2000)&(data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']>5000)&(data['area']<20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']>600)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['tradeMoney']<1000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']<2000)&(data['area']>180)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>50000)&(data['area']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['area']>200)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007') & (data['area']>100)&(data['tradeMoney']<2500)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>200)&(data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>400)&(data['tradeMoney']<15000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']<3000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']>7000)&(data['area']<75)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']>12500)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['area']>400)&(data['tradeMoney']>20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008') & (data['tradeMoney']<2000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00011') & (data['tradeMoney']<10000)&(data['area']>390)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012') & (data['area']>120)&(data['tradeMoney']<5000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']<100)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']>80)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['area']>300)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<1300)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<8000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<1000)&(data['area']>20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']>25000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<20000)&(data['area']>250)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>30000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']<50000)&(data['area']>600)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>50000)&(data['area']>350)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']>4000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']<600)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['area']>165)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012') & (data['tradeMoney']<800)&(data['area']<30)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007') & (data['tradeMoney']<1100)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['tradeMoney']>8000)&(data['area']<80)].index,inplace=True)
    data.loc[(data['region']=='RG00002')&(data['area']>50)&(data['rentType']=='合租'),'rentType']='整租'
    data.loc[(data['region']=='RG00014')&(data['rentType']=='合租')&(data['area']>60),'rentType']='整租'
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>15000)&(data['area']<110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>20000)&(data['area']>110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']<1500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['rentType']=='合租')&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00015') ].index,inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

data_train = cleanData(data_train)

Int64Index([   62,    69,   128,   131,   246,   261,   266,   297,   308,
              313,
            ...
            39224, 39228, 39319, 39347, 39352, 39434, 39563, 41080, 41083,
            41233],
           dtype='int64', length=403)


针对data_train 数据进行特征提取

In [210]:
# 将houseType转为'Room'，'Hall'，'Bath'
def clean2(data_all):
    
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data_all['Room'] = data_all['houseType'].apply(lambda x: Room(x))
    data_all['Hall'] = data_all['houseType'].apply(lambda x: Hall(x))
    data_all['Bath'] = data_all['houseType'].apply(lambda x: Bath(x))
    data_all['Room_Bath'] = (data_all['Bath']+1) / (data_all['Room']+1)
    # 填充租房类型
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['Room'] <= 1), 'rentType'] = '整租'
    # print(data_all.loc[(data_all['rentType']=='未知方式')&(data_all['Room_Bath']>1),'rentType'])
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['Room_Bath'] > 1), 'rentType'] = '合租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['Room'] > 1) & (data_all['area'] < 50), 'rentType'] = '合租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] / data_all['Room'] < 20), 'rentType'] = '合租'
    # data_all.loc[(data_all['rentType']=='未知方式')&(data_all['area']>60),'rentType']='合租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] <= 50) & (data_all['Room'] == 2), 'rentType'] = '合租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] > 60) & (data_all['Room'] == 2), 'rentType'] = '整租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] <= 60) & (data_all['Room'] == 3), 'rentType'] = '合租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] > 60) & (data_all['Room'] == 3), 'rentType'] = '整租'
    data_all.loc[(data_all['rentType'] == '未知方式') & (data_all['area'] >= 100) & (data_all['Room'] > 3), 'rentType'] = '整租'
    
    return data_all
data_train = clean2(data_train)
data_test = clean2(data_test)

In [211]:
# 合并部分配套设施特征
def clean3(data_all):

    data_all['trainsportNum'] = 5 * data_all['subwayStationNum'] / data_all['subwayStationNum'].mean() + data_all['busStationNum'] / \
                                                                                             data_all[
                                                                                                 'busStationNum'].mean()
    data_all['all_SchoolNum'] = 2 * data_all['interSchoolNum'] / data_all['interSchoolNum'].mean() + data_all['schoolNum'] / data_all[
        'schoolNum'].mean() \
                            + data_all['privateSchoolNum'] / data_all['privateSchoolNum'].mean()
    data_all['all_hospitalNum'] = 2 * data_all['hospitalNum'] / data_all['hospitalNum'].mean() + \
                              data_all['drugStoreNum'] / data_all['drugStoreNum'].mean()
    data_all['all_mall'] = data_all['mallNum'] / data_all['mallNum'].mean() + \
                       data_all['superMarketNum'] / data_all['superMarketNum'].mean()
    data_all['otherNum'] = data_all['gymNum'] / data_all['gymNum'].mean() + data_all['bankNum'] / data_all['bankNum'].mean() + \
                       data_all['shopNum'] / data_all['shopNum'].mean() + 2 * data_all['parkNum'] / data_all['parkNum'].mean()

    data_all.drop(['subwayStationNum', 'busStationNum',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
              axis=1, inplace=True)
    # 提升0.0005
    
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return data_all
data_train = clean3(data_train)
data_test = clean3(data_test)

In [213]:
# data_all.drop('houseType', axis=1, inplace=True)

data_all["area"] = data_train["area"].astype(int)

# categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName','region', 'plate']
categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate','cluster']

# 计算统计特征

In [214]:
#计算统计特征
def featureCount(data):
#     train['data_type'] = 0
#     test['data_type'] = 1
#     data = pd.concat([train, test], axis=0, join='outer')
    def feature_count(data, features=[]):
        new_feature = 'count'
        for i in features:
            new_feature += '_' + i
        temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
        data = data.merge(temp, 'left', on=features)
        return data

    data = feature_count(data, ['communityName'])
    data = feature_count(data, ['buildYear'])
    data = feature_count(data, ['totalFloor'])
    data = feature_count(data, ['communityName', 'totalFloor'])
    data = feature_count(data, ['communityName', 'newWorkers'])
    data = feature_count(data, ['communityName', 'totalTradeMoney'])
#     new_train = data[data['data_type'] == 0]
#     new_test = data[data['data_type'] == 1]
#     data.drop('data_type', axis=1, inplace=True)
#     new_test.drop(['data_type'], axis=1, inplace=True)
    return data
    
data_train = featureCount(data_train)
data_test = featureCount(data_test)

# groupby方法生成统计特征

In [215]:
#groupby生成统计特征：mean,std等

def gourpby(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    columns = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
    
    data['price_per_area'] = data.tradeMeanPrice / data.area * 100
    temp = data.groupby('communityName')['price_per_area'].agg(
        {'comm_price_mean': 'mean', 'comm_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
   
    temp = data.groupby('plate')['price_per_area'].agg(
        {'plate_price_mean': 'mean', 'plate_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    data.drop('price_per_area', axis=1, inplace=True)

    temp = data.groupby('plate')['area'].agg({'plate_area_mean': 'mean', 'plate_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    
    temp = data.groupby(['plate'])['buildYear'].agg({'plate_year_mean': 'mean', 'plate_year_std': 'std'})
    data = data.merge(temp, on='plate', how='left')
    data.plate_year_mean = data.plate_year_mean.astype('int')
    data['comm_plate_year_diff'] = data.buildYear - data.plate_year_mean
    data.drop('plate_year_mean', axis=1, inplace=True)

    temp = data.groupby('plate')['trainsportNum'].agg('sum').reset_index(name='plate_trainsportNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['trainsportNum'].agg('sum').reset_index(name='com_trainsportNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['trainsportNum_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                           data['com_trainsportNum'], data['plate_trainsportNum']))
    data = data.drop(['com_trainsportNum', 'plate_trainsportNum'], axis=1)

    temp = data.groupby('plate')['all_SchoolNum'].agg('sum').reset_index(name='plate_all_SchoolNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['all_SchoolNum'].agg('sum').reset_index(name='com_all_SchoolNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data = data.drop(['com_all_SchoolNum', 'plate_all_SchoolNum'], axis=1)

    temp = data.groupby(['communityName', 'plate'])['all_mall'].agg('sum').reset_index(name='com_all_mall')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')

    temp = data.groupby('plate')['otherNum'].agg('sum').reset_index(name='plate_otherNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['otherNum'].agg('sum').reset_index(name='com_otherNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['other_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                   data['com_otherNum'], data['plate_otherNum']))
    data = data.drop(['com_otherNum', 'plate_otherNum'], axis=1)

    temp = data.groupby(['month', 'communityName']).size().reset_index(name='communityName_saleNum')
    data = data.merge(temp, on=['month', 'communityName'], how='left')
    temp = data.groupby(['month', 'plate']).size().reset_index(name='plate_saleNum')
    data = data.merge(temp, on=['month', 'plate'], how='left')

    data['sale_ratio'] = round((data.communityName_saleNum + 1) / (data.plate_saleNum + 1), 3)
    data['sale_newworker_differ'] = 3 * data.plate_saleNum - data.newWorkers
    data.drop(['communityName_saleNum', 'plate_saleNum'], axis=1, inplace=True)

    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test

data_train, data_test = gourpby(data_train, data_test)

# 聚类方法

In [216]:
#聚类
def cluster(train,test):
    from sklearn.mixture import GaussianMixture

    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    col = ['totalFloor',
           'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',

           'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
           'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',

           'landTotalPrice', 'landMeanPrice', 'totalWorkers',
           'newWorkers', 'residentPopulation', 'lookNum',
           'trainsportNum',
           'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']

    # EM
    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']= pd.DataFrame(gmm.fit_predict(data[col]))


    col1 = ['totalFloor','houseDecoration', 'communityName', 'region', 'plate', 'buildYear']
    col2 = ['tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
            'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',
            'landTotalPrice', 'landMeanPrice', 'totalWorkers',
            'newWorkers', 'residentPopulation', 'lookNum',
            'trainsportNum',
            'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']
    for feature1 in col1:
        for feature2 in col2:
        
            temp = data.groupby(['cluster',feature1])[feature2].agg('mean').reset_index(name=feature2+'_'+feature1+'_cluster_mean')
            temp.fillna(0, inplace=True)
       
            data = data.merge(temp, on=['cluster', feature1], how='left')
    
   
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    
    return new_train, new_test

data_train, data_test = cluster(data_train, data_test)   

# log平滑

In [217]:
# 过大量级值取log平滑（针对线性模型有效）
big_num_cols = ['totalTradeMoney','totalTradeArea','tradeMeanPrice','totalNewTradeMoney', 'totalNewTradeArea',
                'tradeNewMeanPrice','remainNewNum', 'supplyNewNum', 'supplyLandArea',
                'tradeLandArea','landTotalPrice','landMeanPrice','totalWorkers','newWorkers',
                'residentPopulation','pv','uv']
for col in big_num_cols:
        data_train[col] = data_train[col].map(lambda x: np.log1p(x))
        data_test[col] = data_test[col].map(lambda x: np.log1p(x))

In [229]:
# data_train = data_train.drop(['houseType'],axis =1)
data_test = data_test.drop(['houseType','tradeMoney'],axis =1)

In [230]:
target_train = data_train.pop('tradeMoney')

KeyError: 'tradeMoney'

In [232]:
#对比特征工程前后线性模型结果情况
test=test.fillna(0)
# Lasso回归
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(data_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(data_train)
y_pred_test=lasso.predict(data_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

训练集结果： 0.7358893296396083
测试集结果： 0.8258261575487295


# 特征选择

# 相关系数法

In [233]:
#相关系数法特征选择
from sklearn.feature_selection import SelectKBest

print(data_train.shape)

sk=SelectKBest(k=150)
new_train=sk.fit_transform(data_train,target_train)
print(new_train.shape)

# 获取对应列索引
select_columns=sk.get_support(indices = True)
# print(select_columns)

# 获取对应列名
# print(test.columns[select_columns])
select_columns_name=data_test.columns[select_columns]
new_test=test[select_columns_name]
print(new_test.shape)
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

(40134, 174)
(40134, 150)
(2469, 150)
训练集结果： 0.7235858160841583
测试集结果： -0.09557046125123114


# Wrapper

In [234]:
# Wrapper

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=160)
rfe.fit(data_train,target_train)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=40, step=1, verbose=0)




select_columns = [f for f, s in zip(data_train.columns, rfe.support_) if s]
print(select_columns)
new_train = data_train[select_columns]
new_test = data_test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

['Bath', 'Hall', 'Room', 'Room_Bath', 'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'area', 'buildYear', 'communityName', 'count_buildYear', 'count_communityName', 'count_communityName_newWorkers', 'count_communityName_totalFloor', 'count_communityName_totalTradeMoney', 'count_totalFloor', 'day', 'houseDecoration', 'houseFloor', 'houseToward', 'landMeanPrice', 'landTotalPrice', 'lookNum', 'month', 'newWorkers', 'otherNum', 'plate', 'pv', 'region', 'remainNewNum', 'rentType', 'residentPopulation', 'saleSecHouseNum', 'supplyLandArea', 'supplyLandNum', 'supplyNewNum', 'totalFloor', 'totalNewTradeArea', 'totalNewTradeMoney', 'totalTradeArea', 'totalTradeMoney', 'totalWorkers', 'tradeLandArea', 'tradeLandNum', 'tradeMeanPrice', 'tradeNewMeanPrice', 'tradeNewNum', 'tradeSecNum', 'trainsportNum', 'uv', 'com_area_mean', 'com_area_std', 'comm_price_mean', 'comm_price_std', 'plate_price_mean', 'plate_price_std', 'plate_area_mean', 'plate_area_std', 'plate_year_std', 'comm_plate_year_diff', 'tr

# Embedded
基于惩罚项的特征选择法  
Lasso(l1)和Ridge(l2)  

In [237]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=5)
ridge.fit(data_train,target_train)

Ridge(alpha=5, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

# 特征系数排序
coefSort = ridge.coef_.argsort()
print(coefSort)


# 特征系数
featureCoefSore=ridge.coef_[coefSort]
print(featureCoefSore)


select_columns = [f for f, s in zip(data_train.columns, featureCoefSore) if abs(s)> 0.0000005 ] 
# 选择绝对值大于0.0000005的特征

new_train = data_train[select_columns]
new_test = data_test[select_columns]
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

[ 63   3 151 136 118 115  44 117   4  81 173 119  78 133 132  27 150  62
  43  60 116 137  37 169 134   2  45  24   8 170  33 161  29 125  18  89
  19  56 121  17  71  40  32  11  20  90  97 126  94  47 107 144  99  87
   9 101 141  84 100  14  72  74  54 162 146  16 164 110  66 124  95  15
 166  77  52 149 113 148  64 147 111  53 106 122  91 163 158  68 127 104
 145 109  73 140  86 165  75 167 112  10  70 160 129  76 131 142 156 120
 102  55 105  93  69 138 130  88  67 128 159  12 123  41  46  92  96  65
 157  61 108 103  98  22  35  42  50 139  13  58  21  51 114  79 143  23
  85  36 168   7  59  57  38 153   5  26  83  80  39  34  49  30  25 155
   1 171  31   6 135 172 154 152  28  82   0  48]
[-6.80563976e+02 -6.49654207e+02 -5.78414536e+02 -4.01981632e+02
 -3.75375567e+02 -3.64567099e+02 -3.59462077e+02 -3.45882054e+02
 -3.10160138e+02 -2.95305836e+02 -2.81415457e+02 -2.81183926e+02
 -2.51099236e+02 -2.44747486e+02 -1.46692041e+02 -1.46410263e+02
 -1.21999598e+02 -1.16784126e+02 

# 基于树模型的特征选择法  
随机森林 平均不纯度减少（mean decrease impurity

In [238]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# 训练随机森林模型，并通过feature_importances_属性获取每个特征的重要性分数。rf = RandomForestRegressor()
rf.fit(data_train,target_train)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), data_train.columns),
             reverse=True))

select_columns = [f for f, s in zip(data_train.columns, rf.feature_importances_) if abs(s)> 0.00005 ] 
# 选择绝对值大于0.00005的特征

new_train = data_train[select_columns]
new_test = data_test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

Features sorted by their score:
[(0.4518, 'area'), (0.1526, 'tradeMeanPrice_plate_cluster_mean'), (0.0605, 'tradeMeanPrice_communityName_cluster_mean'), (0.0288, 'com_area_mean'), (0.027, 'plate_area_mean'), (0.0249, 'plate_area_std'), (0.0232, 'plate_year_std'), (0.0109, 'tradeNewMeanPrice_plate_cluster_mean'), (0.0086, 'totalFloor'), (0.0086, 'comm_plate_year_diff'), (0.0073, 'tradeNewMeanPrice_communityName_cluster_mean'), (0.0068, 'plate_price_mean'), (0.0064, 'buildYear'), (0.0063, 'comm_price_mean'), (0.005, 'day'), (0.0049, 'Room'), (0.0047, 'trainsportNum_communityName_cluster_mean'), (0.0043, 'remainNewNum_communityName_cluster_mean'), (0.0042, 'tradeSecNum_communityName_cluster_mean'), (0.0042, 'sale_ratio'), (0.0039, 'com_all_mall'), (0.0038, 'other_ratio'), (0.0037, 'com_area_std'), (0.0036, 'communityName'), (0.0035, 'trainsportNum_ratio'), (0.0031, 'tradeMeanPrice'), (0.0031, 'count_communityName'), (0.0028, 'trainsportNum'), (0.0028, 'comm_price_std'), (0.0027, 'tradeNew